# Example of Thoth Wet Lab Smart Orchestration SCP Server

This example will use both the Thoth-Plan and Thoth-OP services. Among them, the Thoth-OP service will be automatically invoked within the tools of Thoth-Plan.  
The different code blocks in this Jupyter Notebook have variable dependencies and need to be run sequentially in order.

In [ ]:
import json
import os
from typing import List
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession

# Set server addresses
SCP_WORKFLOW_SERVER_URL = "https://scp.intern-ai.org.cn/api/v1/mcp/32/SCP-Workflow" # SCP Workflow server address
THOTH_PLAN_SERVER_URL = "https://scp.intern-ai.org.cn/api/v1/mcp/19/Thoth-Plan" # Thoth-Plan server address

protocols = []

# Define client class
class ToolsClient:
    def __init__(self, server_url: str):
        self.server_url = server_url
        self.session = None

    async def connect(self):
        """Establish connection and initialize session"""
        print(f"\n{'='*80}")
        print("Connecting to synchronous PCR tool MCP server")
        print(f"{'='*80}")
        print(f"Server address: {self.server_url}")

        try:
            # Establish streamable-http transport connection
            self.transport = streamablehttp_client(
                url=self.server_url, 
                sse_read_timeout=60 * 10,
                headers={
                    "SCP-HUB-API-KEY": "XX"  # replace it with your API Key
                }
            )
            self.read, self.write, self.get_session_id = await self.transport.__aenter__()

            # Create client session
            self.session_ctx = ClientSession(self.read, self.write)
            self.session = await self.session_ctx.__aenter__()

            # Initialize session
            await self.session.initialize()
            session_id = self.get_session_id()

            print(f"✓ Connection successful")
            print(f"✓ Session ID: {session_id}")
            print(f"{'='*80}\n")
            return True

        except Exception as e:
            print(f"✗ Connection failed: {e}")
            import traceback
            traceback.print_exc()
            return False

    async def disconnect(self):
        """Disconnect"""
        try:
            if self.session:
                await self.session_ctx.__aexit__(None, None, None)
            if hasattr(self, 'transport'):
                await self.transport.__aexit__(None, None, None)
            print("\n✓ Disconnected\n")
        except Exception as e:
            print(f"✗ Error disconnecting: {e}")

    async def list_tools(self):
        """List all available tools"""
        print(f"\n{'='*80}")
        print("Listing all available tools")
        print(f"{'='*80}")

        try:
            tools_list = await self.session.list_tools()
            print(f"\nAvailable tools (Total {len(tools_list.tools)}):\n")

            for i, tool in enumerate(tools_list.tools, 1):
                print(f"{i:2d}. {tool.name}")
                if tool.description:
                    # Only display first line of description
                    desc_line = tool.description
                    print(f"    {desc_line}")

            print(f"\n✓ Tool list retrieved successfully")
            return tools_list.tools

        except Exception as e:
            print(f"✗ Failed to list tools: {e}")
            return []

    def _parse_result(self, result):
        """Parse MCP tool call result"""
        try:
            if hasattr(result, 'content') and result.content:
                content = result.content[0]
                if hasattr(content, 'text'):
                    return json.loads(content.text)
            return str(result)
        except Exception as e:
            try:
                return result.content[0].text
            except Exception as e2:
                return {"error": f"Failed to parse result: {e}\t{e2}", "raw": str(result)}
        

async def upload_file_via_curl(upload_url: str, file_path: str = "./test.pdf"):
    """Upload file to pre-signed URL using curl command"""
    import subprocess
    
    if not os.path.exists(file_path):
        print(f"✗ File does not exist: {file_path}")
        return False
    
    try:
        # Use curl to upload file
        curl_cmd = f"curl -X PUT '{upload_url}' --upload-file {file_path}"
        print(f"Executing upload command: {curl_cmd}")
        
        result = subprocess.run(curl_cmd, shell=True, capture_output=True, text=True)
        
        if result.returncode == 0:
            print("✓ File uploaded successfully")
            return True
        else:
            print(f"✗ File upload failed:")
            print(f"  stderr: {result.stderr}")
            print(f"  stdout: {result.stdout}")
            return False
            
    except Exception as e:
        print(f"✗ Error uploading file: {e}")
        return False

## Generate Protocol Based on User-Uploaded PDF Files

In [2]:
async def upload_and_generate_protocol() -> str:
    client_file = ToolsClient(server_url=SCP_WORKFLOW_SERVER_URL)
    if not await client_file.connect():
        print("connection failed")
        return
    
    print("Uploading file to the cloud...")
    file_result = await client_file.session.call_tool(
        "generate_presigned_url",
        arguments={
            "key": "test.pdf",
            "expires_seconds": 3600
        }
    )
    file_result_json = client_file._parse_result(file_result)
    
    if "error" in file_result_json:
        print(f"✗ Failed to get presigned URL: {file_result_json}")
        await client_file.disconnect()
        return
        
    upload_url = file_result_json["upload"]["url"]
    download_url = file_result_json["download"]["url"]
    
    print(f"Upload URL: {upload_url}")
    print(f"Download URL: {download_url}")
    
    success = await upload_file_via_curl(upload_url, "./test.pdf")
    
    await client_file.disconnect()

    if not success:
        print("✗ File upload failed, exiting the program")
        return
    
    
    print("Extracting a protocol from the PDF...")
    client = ToolsClient(server_url=THOTH_PLAN_SERVER_URL)
    if not await client.connect():
        print("connection failed")
        return
    
    result = await client.session.call_tool(
        "extract_protocol_from_pdf",
        arguments={
            "pdf_url": download_url
        }
    )
    protocol = client._parse_result(result)
    
    await client.disconnect()
    print("=" * 80)
    print("Extracted protocol from PDF:")
    print("=" * 80)
    print(protocol)
    
    return protocol


protocol_from_pdf = await upload_and_generate_protocol()
protocols.append(protocol_from_pdf)


Connecting to synchronous PCR tool MCP server
Server address: https://scp.intern-ai.org.cn/api/v1/mcp/32/SCP-Workflow
✓ Connection successful
✓ Session ID: 04d0f3d9-95fa-4df3-bbe9-0924b06c3149

Uploading file to the cloud...
Upload URL: https://ai4s-scp.oss-cn-shanghai.aliyuncs.com/2025/12/23/test_065923_c8f8e860.pdf?x-oss-signature-version=OSS4-HMAC-SHA256&x-oss-date=20251223T065923Z&x-oss-expires=3599&x-oss-credential=LTAI5t8mY1oSnxuXrvuLiSkC%2F20251223%2Fcn-shanghai%2Foss%2Faliyun_v4_request&x-oss-signature=fe743b0b2add481d09247564d839316fd89efc208b553e577d9df13786be380f
Download URL: https://ai4s-scp.oss-cn-shanghai.aliyuncs.com/2025/12/23/test_065923_c8f8e860.pdf?x-oss-signature-version=OSS4-HMAC-SHA256&x-oss-date=20251223T065923Z&x-oss-expires=3599&x-oss-credential=LTAI5t8mY1oSnxuXrvuLiSkC%2F20251223%2Fcn-shanghai%2Foss%2Faliyun_v4_request&x-oss-signature=eb23dd52fe5120e5c6cfe0881c3a5f3607417079d8966ded06f3e1893c1ac8e1
Executing upload command: curl -X PUT 'https://ai4s-scp.oss-

## Generate Protocols Based on User-provided Requirements

In [3]:
# Multiple user-provided requirements
prompts = [
    # 20 minute PCR enzymatic cleanup procedure
    "Could you describe the full set of laboratory steps used in the patch based colony PCR protocol for screening Phaeodactylum tricornutum?",
    
    # real time PCR assay for detecting viable Dichelobacter nodosus with PMAxx
    "I would like to know every step involved in the real time PCR assay for detecting viable Dichelobacter nodosus with PMAxx. Can you provide the full protocol?",
    
    # highly sensitive detection of the HV69/70 deletion
    "Could you provide the full set of laboratory steps used in the Modified Zhen et al. SARS-CoV-2 Spike-Gene qRT-PCR assay for highly sensitive detection of the HV69/70 deletion?",
    
    # patch based colony PCR protocol for screening Phaeodactylum tricornutum
    "Could you describe the full set of laboratory steps used in the patch based colony PCR protocol for screening Phaeodactylum tricornutum?",
    
    # perform the Illumina TruSeq library quantification using the qPCR probe method
    "How do you perform the Illumina TruSeq library quantification using the qPCR probe method? Please describe the complete protocol.",
    
    # converting RNA to cDNA and then carrying out the RT PCR procedure
    "Could you outline the entire workflow for converting RNA to cDNA and then carrying out the RT PCR procedure, including all steps in the protocol?",
    
    # Ultra Rapid PCR sequencing
    "Could you explain the full experimental process for carrying out the Ultra Rapid PCR sequencing protocol?",
    
    # Rubella virus real time RT PCR assay
    "Please explain the complete procedure for performing the Rubella virus real time RT PCR assay, including every step from start to finish.",
    
    # standard Colony PCR
    "What is the complete procedure for performing a standard Colony PCR experiment?",
    
    # SARS-CoV-2 McGill ARTIC PCR protocol
    "Can you describe all steps required to run the SARS-CoV-2 McGill ARTIC PCR protocol using 2.5 µl RT with V3-only primers and LA1?"
]


async def generate_protocol(prompts: List[str]):
    generated_protocols = []
    
    client = ToolsClient(server_url=THOTH_PLAN_SERVER_URL)
    if not await client.connect():
        print("connection failed")
        return
    
    for prompt_id, prompt in enumerate(prompts):
        result = await client.session.call_tool(
            "protocol_generation",
            arguments={
                "user_prompt": prompt
            }
        )
        protocol = client._parse_result(result)
        
        print("=" * 80)
        print(f"USER QUERY #{prompt_id + 1}: {prompt}")
        print("GENERATED PROTOCOL:")
        print(protocol)
        
        generated_protocols.append(protocol)
        
    await client.disconnect()
        
    return generated_protocols
    

protocols.extend(await generate_protocol(prompts))
    


Connecting to synchronous PCR tool MCP server
Server address: https://scp.intern-ai.org.cn/api/v1/mcp/19/Thoth-Plan
✓ Connection successful
✓ Session ID: 75429e86-0fdf-43b5-a5ee-589753c934a4

USER QUERY #1: Could you describe the full set of laboratory steps used in the patch based colony PCR protocol for screening Phaeodactylum tricornutum?
GENERATED PROTOCOL:
1. Prepare 5’/3’ primer working stock by diluting each primer 1:10 into a single clean Eppendorf tube from an initial 100 uM stock (ex: 16 uL ddH20 + 2 uL 5’ primer + 2 uL 3’ primer in 20 uL total). Each PCR reaction will require 1 uL of primer mix, but due to pipetting error, it is good practice to generate at least 5-10 extra samples. Final concentration of each primer in the mix in 500 nM.
2. To set up the PCR master mix, use the table below: #colonies to screen/#rxns to prep Single reaction 12/16 48/56 96/112 5’/3’ mix 1 16 56 112 2x OneTaq Master Mix 10 160 560 1120 ddH O 9 144 504 1008 Final volume (uL) 20 320 1120 2240; 

## Convert Protocols to Executable JSON and Send to Thoth-OP for Execution


In [4]:
async def execute_json(protocols: List[str]):
    client = ToolsClient(server_url=THOTH_PLAN_SERVER_URL)
    if not await client.connect():
        print("connection failed")
        return
    
    for protocol_id, protocol in enumerate(protocols):
        prot_desc = f"the protocol generated given user query #{protocol_id}" if protocol_id > 0 else "the protocol generated given user-uploaded PDF file"
        print(f"Converting the protocol to an executable JSON for {prot_desc}...")
        result = await client.session.call_tool(
            "generate_executable_json",
            arguments={
                "protocol": protocol
            }
        )
        executable_json = client._parse_result(result)
        
        print(f"Executing the JSON for {prot_desc}...")
        result = await client.session.call_tool(
            "execute_json",
            arguments={
                "executable_json": json.dumps(executable_json)
            }
        )
        execution_info = client._parse_result(result)
        print("Execution information:")
        print(execution_info)
        print("=" * 20)
        
    await client.disconnect()
    
await execute_json(protocols)


Connecting to synchronous PCR tool MCP server
Server address: https://scp.intern-ai.org.cn/api/v1/mcp/19/Thoth-Plan
✓ Connection successful
✓ Session ID: 1433e3f1-89c1-464d-9fba-d678020e7b19

Converting the protocol to an executable JSON for the protocol generated given user-uploaded PDF file...
Executing the JSON for the protocol generated given user-uploaded PDF file...
Execution information:
1. Preparing 5'/3' primer working stock (volume: 20:microliter) at 500 nM each primer in primer_mix_tube.
2. Preparing PCR master mix (volume: 20:microliter) in pcr_master_mix_tube.
3. Transferring from pcr_master_mix_tube to pcr_tube, volume: 20:microliter.
4. Transferring from pcr_master_mix_tube to positive_control_tube, volume: 19:microliter.
5. Adding purified plasmid (0.5-1.0 ng/uL) to positive_control_tube, volume: 1:microliter.
6. Transferring from pcr_master_mix_tube to negative_control_tube, volume: 19:microliter.
7. Adding ddH2O to negative_control_tube, volume: 1:microliter.
8. Perf